In [1]:
import numpy as np
import cv2
import scipy
from scipy import linalg
import io
import os 
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12,9)


In [2]:
path = "data"
filenames = ["3d_1.txt", "3d_2.txt"]

pointsets = []
for filename in filenames:
    file_3d = os.path.join(path,filename)
    delimiter3d = ' '

    pointset = {}
    with open(file_3d) as f3d:
        for l3d in f3d:
            l3d_split = l3d.split(delimiter3d)
            if len(l3d_split)==4:
                pointset[l3d_split[0]] = np.array([float(num) for num in l3d_split[1:]])
    pointsets.append(pointset)

In [58]:
for pointset_tgt in pointsets:
    for pointset_src in pointsets:
        p_tgt = []
        p_src = []
        for key in list(set(pointset_tgt.keys()) & set(pointset_src.keys())):
            p_tgt.append(pointset_tgt[key])
            p_src.append(pointset_src[key])
        p_tgt = np.array(p_tgt).astype('float32')
        p_src = np.array(p_src).astype('float32')
        p_tgt_mean = p_tgt.mean(axis=0)
        p_src_mean = p_src.mean(axis=0)
        p_tgt_ = p_tgt - p_tgt_mean
        p_src_ = p_src - p_src_mean
        H = np.dot(p_src_[:,:2].T, p_tgt_[:,:2])
        u,s,vh = np.linalg.svd(H)
        R = np.dot(vh.T, u.T)
        R = np.hstack( (np.vstack((R,np.array([0,0]))), np.array([0,0,1]).reshape(3,1)) )
        t = p_tgt_mean - np.dot(R , p_src_mean)
        p_src_aligned = np.dot(R , p_src.T).T + t
        print np.max(p_tgt - p_src_aligned)
        print R


2.270937056891853e-05
[[0.99999994 0.         0.        ]
 [0.         0.99999994 0.        ]
 [0.         0.         1.        ]]
4.12907710216632
[[ 0.9995786   0.02902823  0.        ]
 [-0.02902823  0.9995786   0.        ]
 [ 0.          0.          1.        ]]
2.1132280506521255
[[ 0.9995786  -0.02902823  0.        ]
 [ 0.02902823  0.9995786   0.        ]
 [ 0.          0.          1.        ]]
0.0
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [57]:
pointset_tgt = pointsets[0]
pointset_src = pointsets[1]
p_tgt = []
p_src = []
for key in list(set(pointset_tgt.keys()) & set(pointset_src.keys())):
    p_tgt.append(pointset_tgt[key])
    p_src.append(pointset_src[key])
p_tgt = np.array(p_tgt).astype('float32')
p_src = np.array(p_src).astype('float32')
p_tgt_mean = p_tgt.mean(axis=0)
p_src_mean = p_src.mean(axis=0)
p_tgt_ = p_tgt - p_tgt_mean
p_src_ = p_src - p_src_mean
H = np.dot(p_src_[:,:2].T, p_tgt_[:,:2])
u,s,vh = np.linalg.svd(H)
R = np.dot(vh.T, u.T)
R = np.hstack( (np.vstack((R,np.array([0,0]))), np.array([0,0,1]).reshape(3,1)) )
t = p_tgt_mean - np.dot(R , p_src_mean)
p_src_aligned = np.dot(R , p_src.T).T + t
print np.max(p_tgt - p_src_aligned)
print R


4.12907710216632
[[ 0.9995786   0.02902823  0.        ]
 [-0.02902823  0.9995786   0.        ]
 [ 0.          0.          1.        ]]


In [59]:
p_tgt_mean

array([-100.423065, -226.89232 ,  130.7846  ], dtype=float32)

In [11]:
p_tgt

array([[ -1.5, -11.7, 215.5],
       [-94.6, -17.7,   0. ],
       [ -1.5, -11.7, 209.1],
       [  0.5,   0.5,   0. ]], dtype=float32)

In [51]:
R

array([[ 0.93589294,  0.35220277,  0.0075932 ],
       [ 0.35169208, -0.9328521 , -0.07810004],
       [ 0.0204237 , -0.07576375,  0.9969166 ]], dtype=float32)

array([[ 0.99991834, -0.01234196, -0.00328646,  0.        ],
       [ 0.01224281,  0.99951458, -0.0286483 ,  0.        ],
       [ 0.00363845,  0.02860573,  0.99958414,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [39]:
 np.array([[0,0,0,1]])

array([[0, 0, 0, 1]])

In [30]:
R

array([[ 0.99991834, -0.01234196, -0.00328646],
       [ 0.01224281,  0.9995146 , -0.0286483 ],
       [ 0.00363845,  0.02860573,  0.99958414]], dtype=float32)